#FactorCL on Synthetic Dataset

In [1]:
!git clone https://github.com/pliang279/FactorCL

Cloning into 'FactorCL'...
remote: Enumerating objects: 146, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 146 (delta 11), reused 0 (delta 0), pack-reused 116
Receiving objects: 100% (146/146), 317.87 KiB | 9.08 MiB/s, done.
Resolving deltas: 100% (68/68), done.


In [2]:
%cd FactorCL

/content/FactorCL


In [3]:
from Synthetic.dataset import*
from synthetic_model import*
from torch.utils.data import DataLoader
from sklearn.linear_model import LogisticRegression

In [4]:
hidden_dim=512
embed_dim=128

In [5]:

# Define custom dimensions of features and labels
feature_dim_info = dict()
label_dim_info = dict()

intersections = get_intersections(num_modalities=2)

feature_dim_info['12'] = 10
feature_dim_info['1'] = 6
feature_dim_info['2'] = 6

label_dim_info['12'] = 10
label_dim_info['1'] = 6
label_dim_info['2'] = 6

print(intersections)
print(feature_dim_info)
print(label_dim_info)

# Get datasets
total_data, total_labels, total_raw_features = generate_data(30000, 2, feature_dim_info, label_dim_info)
total_labels = get_labels(label_dim_info, total_raw_features)

dataset = MultimodalDataset(total_data, total_labels)

# Dataloader
batch_size = 256
num_data = total_labels.shape[0]
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [int(0.8*num_data), num_data-int(0.8*num_data)])

train_loader = DataLoader(train_dataset, shuffle=True, drop_last=True,
                            batch_size=batch_size)
test_loader = DataLoader(test_dataset, shuffle=False, drop_last=False,
                            batch_size=batch_size)
data_loader = DataLoader(dataset, shuffle=False, drop_last=False,
                            batch_size=batch_size)


['1', '2', '12']
{'12': 10, '1': 6, '2': 6}
{'12': 10, '1': 6, '2': 6}


##Linear Probing

###FactorCL-SUP

In [ ]:
factorcl_sup = FactorCLSUP(A_dim, B_dim, 20, hidden_dim, embed_dim).cuda()
train_sup_model(factorcl_sup, train_loader, dataset, num_epoch=10, num_club_iter=1)
factorcl_sup.eval()

iter:  0  i_batch:  0  loss:  9.623612277209759e-05
iter:  1  i_batch:  0  loss:  -3.0933303833007812
iter:  2  i_batch:  0  loss:  -5.884997367858887
iter:  3  i_batch:  0  loss:  -7.007236480712891
iter:  4  i_batch:  0  loss:  -7.123583793640137
iter:  5  i_batch:  0  loss:  -7.228031158447266
iter:  6  i_batch:  0  loss:  -7.025860786437988
iter:  7  i_batch:  0  loss:  -7.642902374267578
iter:  8  i_batch:  0  loss:  -8.002883911132812
iter:  9  i_batch:  0  loss:  -7.967354774475098


FactorCLSUP(
  (backbones): ModuleList(
    (0-1): 2 x Sequential(
      (0): Linear(in_features=100, out_features=512, bias=True)
      (1): ReLU()
      (2): Linear(in_features=512, out_features=512, bias=True)
      (3): ReLU()
      (4): Linear(in_features=512, out_features=512, bias=True)
      (5): ReLU()
      (6): Linear(in_features=512, out_features=128, bias=True)
    )
  )
  (linears_infonce_x1x2): ModuleList(
    (0-1): 2 x Sequential(
      (0): Linear(in_features=128, out_features=128, bias=True)
      (1): ReLU()
      (2): Linear(in_features=128, out_features=128, bias=True)
      (3): ReLU()
      (4): Linear(in_features=128, out_features=128, bias=True)
    )
  )
  (linears_club_x1x2_cond): ModuleList(
    (0-1): 2 x Sequential(
      (0): Linear(in_features=128, out_features=128, bias=True)
      (1): ReLU()
      (2): Linear(in_features=128, out_features=128, bias=True)
      (3): ReLU()
      (4): Linear(in_features=128, out_features=128, bias=True)
    )
  )
  (li

In [ ]:
# Simple evaluation using linear logistic regression model

# Embeddings
train_embeds = factorcl_sup.get_embedding(torch.stack(train_dataset[:][:-1]).cuda()).detach().cpu().numpy()
train_labels = train_dataset[:][-1].detach().cpu().numpy()

test_embeds = factorcl_sup.get_embedding(torch.stack(test_dataset[:][:-1]).cuda()).detach().cpu().numpy()
test_labels = test_dataset[:][-1].detach().cpu().numpy()

# Train Logistic Classifier
clf = LogisticRegression(max_iter=200).fit(train_embeds, train_labels)
score = clf.score(test_embeds, test_labels)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
score

0.9863333333333333

###FactorCL-SSL

In [ ]:
factorcl_ssl = FactorCLSSL(A_dim, B_dim, hidden_dim, embed_dim).cuda()
train_sup_model(factorcl_ssl, train_loader, dataset, num_epoch=10, num_club_iter=1)
factorcl_ssl.eval()

iter:  0  i_batch:  0  loss:  3.082677721977234e-06
iter:  1  i_batch:  0  loss:  -7.57092809677124
iter:  2  i_batch:  0  loss:  -8.953313827514648
iter:  3  i_batch:  0  loss:  -10.78848648071289
iter:  4  i_batch:  0  loss:  -12.652485847473145
iter:  5  i_batch:  0  loss:  -14.410904884338379
iter:  6  i_batch:  0  loss:  -14.283191680908203
iter:  7  i_batch:  0  loss:  -14.439704895019531
iter:  8  i_batch:  0  loss:  -15.733597755432129
iter:  9  i_batch:  0  loss:  -15.78852653503418


FactorCLSSL(
  (backbones): ModuleList(
    (0): Sequential(
      (0): Linear(in_features=100, out_features=512, bias=True)
      (1): ReLU()
      (2): Linear(in_features=512, out_features=512, bias=True)
      (3): ReLU()
      (4): Linear(in_features=512, out_features=512, bias=True)
      (5): ReLU()
      (6): Linear(in_features=512, out_features=128, bias=True)
    )
    (1): Sequential(
      (0): Linear(in_features=100, out_features=512, bias=True)
      (1): ReLU()
      (2): Linear(in_features=512, out_features=512, bias=True)
      (3): ReLU()
      (4): Linear(in_features=512, out_features=512, bias=True)
      (5): ReLU()
      (6): Linear(in_features=512, out_features=128, bias=True)
    )
  )
  (linears_infonce_x1x2): ModuleList(
    (0): Sequential(
      (0): Linear(in_features=128, out_features=128, bias=True)
      (1): ReLU()
      (2): Linear(in_features=128, out_features=128, bias=True)
      (3): ReLU()
      (4): Linear(in_features=128, out_features=128, bias=T

In [ ]:
# Simple evaluation using linear logistic regression model

# Embeddings
train_embeds = factorcl_ssl.get_embedding(torch.stack(train_dataset[:][:-1]).cuda()).detach().cpu().numpy()
train_labels = train_dataset[:][-1].detach().cpu().numpy()

test_embeds = factorcl_ssl.get_embedding(torch.stack(test_dataset[:][:-1]).cuda()).detach().cpu().numpy()
test_labels = test_dataset[:][-1].detach().cpu().numpy()

# Train Logistic Classifier
clf = LogisticRegression(max_iter=200).fit(train_embeds, train_labels)
score = clf.score(test_embeds, test_labels)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
score

0.9716666666666667

###SimCLR

In [ ]:
# set use_label=True for SupCon model
simclr_model = SupConModel(A_dim, B_dim, hidden_dim, embed_dim, use_label=False).cuda()
simclr_optim = optim.Adam(simclr_model.parameters(), lr=lr)
train_supcon(simclr_model, train_loader, simclr_optim, num_epoch=20)
simclr_model.eval()

iter:  0  i_batch:  0  loss:  11.985010147094727
iter:  1  i_batch:  0  loss:  5.873631477355957
iter:  2  i_batch:  0  loss:  4.937842845916748
iter:  3  i_batch:  0  loss:  4.577206611633301
iter:  4  i_batch:  0  loss:  3.8152596950531006
iter:  5  i_batch:  0  loss:  2.4593682289123535
iter:  6  i_batch:  0  loss:  1.589794635772705
iter:  7  i_batch:  0  loss:  1.177107810974121
iter:  8  i_batch:  0  loss:  0.9892462491989136
iter:  9  i_batch:  0  loss:  0.9151424169540405
iter:  10  i_batch:  0  loss:  0.809130072593689
iter:  11  i_batch:  0  loss:  0.6216927766799927
iter:  12  i_batch:  0  loss:  0.600822925567627
iter:  13  i_batch:  0  loss:  0.5172191858291626
iter:  14  i_batch:  0  loss:  0.5168259143829346
iter:  15  i_batch:  0  loss:  0.43819260597229004
iter:  16  i_batch:  0  loss:  0.5183995962142944
iter:  17  i_batch:  0  loss:  0.32648688554763794
iter:  18  i_batch:  0  loss:  0.42932015657424927
iter:  19  i_batch:  0  loss:  0.5603318810462952


SupConModel(
  (encoder_x1): Sequential(
    (0): Linear(in_features=100, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=512, bias=True)
    (5): ReLU()
    (6): Linear(in_features=512, out_features=128, bias=True)
  )
  (encoder_x2): Sequential(
    (0): Linear(in_features=100, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=512, bias=True)
    (5): ReLU()
    (6): Linear(in_features=512, out_features=128, bias=True)
  )
  (projection_x1): Sequential(
    (0): Linear(in_features=128, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=128, bias=True)
  )
  (projection_x2): Sequential(
    (0): Linear(in_features=128, out_features=1

In [ ]:
train_embeds = simclr_model.get_embedding(torch.stack(train_dataset[:][:-1]).cuda()).detach().cpu().numpy()
train_labels = train_dataset[:][-1].detach().cpu().numpy()

# Embeddings
test_embeds = simclr_model.get_embedding(torch.stack(test_dataset[:][:-1]).cuda()).detach().cpu().numpy()
test_labels = test_dataset[:][-1].detach().cpu().numpy()

# Train Logistic Classifier
clf = LogisticRegression(max_iter=200).fit(train_embeds, train_labels)
score = clf.score(test_embeds, test_labels)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
score

0.9555

#MI Estimation

We can estimate the MI between any two sets of vectors using the Probablistic Classifier approach proposed in [Neural Methods for Point-wise Dependency Estimation](https://arxiv.org/abs/2006.05553)

In [10]:
# Simple dataset for constructing paired data

class PairedDataset(Dataset):
  def __init__(self, data_A, data_B):
    self.data_A = data_A.float()
    self.data_B = data_B.float()

  def __len__(self):
    return self.data_A.shape[0]

  def __getitem__(self, idx):
    return self.data_A[idx], self.data_B[idx]

  def get_dims(self):
    return self.data_A.shape[1], self.data_B.shape[1]


def train_critic_paired(critic_model, train_loader, opt_crit, num_epoch=80):
    MIs = []
    for _iter in range(num_epoch):
        for i_batch, (A_batch, B_batch) in enumerate(train_loader):
            opt_crit.zero_grad()

            x_batch = A_batch.cuda()
            y_batch = B_batch.cuda()

            scores = critic_model(x_batch, y_batch)
            MIs.append(probabilistic_classifier_eval(scores))

            negative_loss = probabilistic_classifier_obj(scores)
            loss = -negative_loss

            loss.backward()
            opt_crit.step()

            if i_batch%100 == 0:
                print('iter: ', _iter, ' i_batch: ', i_batch, ' negative_loss: ', negative_loss.item())

    return MIs


def eval_MI_paired(critic_model, test_loader):
    MIs = []

    for i_batch, (A_batch, B_batch) in enumerate(test_loader):

        x_batch = A_batch.cuda()
        y_batch = B_batch.cuda()

        scores = critic_model(x_batch, y_batch)
        MIs.append(probabilistic_classifier_eval(scores))

    MI = torch.stack(MIs).mean()
    print(MI.item())
    return MI.item()


In [8]:
layers = 1
activation = 'relu'
lr = 1e-4

In [9]:
data_A = train_dataset[:][0]
data_B = train_dataset[:][1]

mi_dataset = PairedDataset(data_A, data_B)
mi_loader = DataLoader(mi_dataset,
                                 shuffle=True,
                                 batch_size=batch_size)

dim1, dim2 = mi_dataset.get_dims()
critic_model = SeparableCritic(x1_dim=dim1, x2_dim=dim2, hidden_dim=hidden_dim,
                              embed_dim=embed_dim, layers=layers, activation=activation).cuda()

opt_crit = optim.Adam(critic_model.parameters(), lr=lr)

train_critic_paired(critic_model, mi_loader, opt_crit, num_epoch=30)
MI = eval_MI_paired(critic_model, mi_loader)




iter:  0  i_batch:  0  negative_loss:  -0.6815298795700073
iter:  1  i_batch:  0  negative_loss:  -0.02864418551325798
iter:  2  i_batch:  0  negative_loss:  -0.025951214134693146
iter:  3  i_batch:  0  negative_loss:  -0.025350451469421387
iter:  4  i_batch:  0  negative_loss:  -0.022115405648946762
iter:  5  i_batch:  0  negative_loss:  -0.018267907202243805
iter:  6  i_batch:  0  negative_loss:  -0.014728059060871601
iter:  7  i_batch:  0  negative_loss:  -0.012674244120717049
iter:  8  i_batch:  0  negative_loss:  -0.008447285741567612
iter:  9  i_batch:  0  negative_loss:  -0.007021321449428797
iter:  10  i_batch:  0  negative_loss:  -0.005306956823915243
iter:  11  i_batch:  0  negative_loss:  -0.004481859505176544
iter:  12  i_batch:  0  negative_loss:  -0.0029017655178904533
iter:  13  i_batch:  0  negative_loss:  -0.002666341606527567
iter:  14  i_batch:  0  negative_loss:  -0.002147895749658346
iter:  15  i_batch:  0  negative_loss:  -0.0017188391648232937
iter:  16  i_batch: